# Transformer Block (LLM)
This notebook implements a single Transformer block used in autoregressive large language models (LLMs).
It contains implementations and explanations for: `LayerNorm`, `GELU`, `FeedForward`, `MultiHeadAttention`, and the `TransformerBlock` wrapper.
The following cells include concise descriptions of each component and a flowchart illustrating data flow.

## Imports and Setup
These imports provide the neural-net primitives and tensor operations used throughout the block implementations.

In [1]:
import torch.nn as nn
import torch

## Model Configuration
A small config dict used to parameterize dimensions, heads, and dropout.
- `vocab_size`: tokenizer vocabulary size
- `context_length`: maximum sequence length (causal window)
- `emb_dim`: embedding / hidden dimension
- `n_heads`: number of attention heads
- `drop_rate`: dropout probability
- `qkv_bias`: whether linear projections use a bias term

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

### LayerNorm
Layer normalization applied across the embedding dimension: it stabilizes training by normalizing each token's feature vector. This implementation keeps learnable `scale` and `shift` parameters and uses a small `eps` to avoid division by zero.

In [3]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

### GELU Activation
Gaussian Error Linear Unit (GELU) is used as the non-linearity in many transformer feed-forward layers. This cell provides a numerically stable approximation implemented with `tanh`.

In [4]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

### FeedForward Network (MLP)
A two-layer MLP with an expansion factor (commonly 4x) and a GELU activation. Operates independently on each token (position-wise).

In [5]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

### Multi-Head Self-Attention
This implementation projects inputs to queries/keys/values, splits into `num_heads`, computes scaled dot-product attention with a causal mask (upper-triangular), applies dropout to attention weights, then concatenates head outputs and applies a final linear projection.
Note on shapes: input `(b, T, d_in)` -> projected `(b, T, d_out)` -> reshaped `(b, num_heads, T, head_dim)` for attention computations. The mask prevents a token attending to future positions.

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

### Transformer Block (Residual + Norm)
The `TransformerBlock` composes attention and feed-forward sublayers with pre-layer normalization, residual connections, and dropout on the shortcuts. The sequence is:
1. `x -> LayerNorm -> MultiHeadAttention -> Dropout -> + residual`
2. `x -> LayerNorm -> FeedForward -> Dropout -> + residual`
This ordering (pre-norm) is common in modern transformer variants and improves training stability for deep stacks.

In [7]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

## Architecture Flowchart
The flowchart below sketches the data path inside a single transformer block. The mermaid diagram is included; if your notebook environment supports Mermaid rendering (or the `mermaid` extension), it will display visually. Otherwise, a simple ASCII diagram is provided below.
```mermaid
flowchart TD
  Input["Input tokens / Embeddings (b, T, d)"] --> LN1["LayerNorm 1"]
  LN1 --> MHA["Multi-Head Attention"]
  MHA --> Add1["Add & Residual"]
  Add1 --> LN2["LayerNorm 2"]
  LN2 --> FF["FeedForward (MLP)"]
  FF --> Add2["Add & Residual -> Output"]
  Add2 --> Output["Output (b, T, d)"]
```
**ASCII fallback:**
```
Input (b,T,d) -> LayerNorm -> MultiHeadAttention -> +residual -> LayerNorm -> FeedForward -> +residual -> Output (b,T,d)
```